<a href="https://colab.research.google.com/github/aderdouri/ql_web_app/blob/master/ql_notebooks/bermudanswaption.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install QuantLib-Python

In [ ]:
import QuantLib as ql
import unittest

# Helper for preconditions (like if_speed(Fast) in Boost Test)
# This is a simple placeholder; a real test suite might use test markers or env vars.
def if_speed(level):
    def decorator(func):
        # For this translation, we'll assume all tests run.
        # If specific skipping logic is needed, unittest.skipIf could be used.
        return func
    return decorator

class BermudanSwaptionTests(unittest.TestCase):

    class CommonVars:
        def __init__(self):
            # Term structure handle (must be initialized before index)
            self.termStructure = ql.RelinkableYieldTermStructureHandle()

            # Index - Euribor6M uses TARGET calendar by default if not specified otherwise
            # The handle passed to the index constructor will be used by the index.
            self.index = ql.Euribor6M(self.termStructure)

            # Global data, calendar from index
            self.calendar = self.index.fixingCalendar()
            self.today = self.calendar.adjust(ql.Date.todaysDate()) # Will be overridden in tests
            self.settlementDays = 2
            # settlement can also be overridden. Initialize for consistency.
            self.settlement = self.calendar.advance(self.today, self.settlementDays, ql.Days)

            # Underlying swap parameters
            self.startYears = 1
            self.length = 5
            self.type = ql.Swap.Payer
            self.nominal = 1000.0
            self.fixedConvention = ql.Unadjusted
            self.floatingConvention = ql.ModifiedFollowing
            self.fixedFrequency = ql.Annual
            self.floatingFrequency = ql.Semiannual
            self.fixedDayCount = ql.Thirty360(ql.Thirty360.BondBasis)
            # self.index is already initialized above
            # self.settlementDays is already initialized above

        def makeSwap(self, fixedRate):
            start = self.calendar.advance(self.settlement, self.startYears, ql.Years)
            maturity = self.calendar.advance(start, self.length, ql.Years)
            fixedSchedule = ql.Schedule(start, maturity,
                                        ql.Period(self.fixedFrequency),
                                        self.calendar,
                                        self.fixedConvention,
                                        self.fixedConvention,
                                        ql.DateGeneration.Forward, False)
            floatSchedule = ql.Schedule(start, maturity,
                                        ql.Period(self.floatingFrequency),
                                        self.calendar,
                                        self.floatingConvention,
                                        self.floatingConvention,
                                        ql.DateGeneration.Forward, False)

            # The termStructure handle passed to DiscountingSwapEngine is the same one
            # held by self.index, ensuring consistency if it's relinked.
            swap = ql.VanillaSwap(self.type, self.nominal,
                                  fixedSchedule, fixedRate, self.fixedDayCount,
                                  floatSchedule, self.index, 0.0,
                                  self.index.dayCounter())
            swap.setPricingEngine(ql.DiscountingSwapEngine(self.termStructure))
            return swap

    def testCachedValues(self):
        print("Testing Bermudan swaption with HW model against cached values...")

        # Store original setting and restore it later to keep test idempotent
        original_par_coupon_setting = ql.IborCoupon.Settings.instance().usingAtParCoupons()
        # The test doesn't change this setting, it just reads it.
        # For full C++ parity where the setting might be changed by other tests,
        # one might want to save/restore, but here we assume it's at its default or QL's current state.
        usingAtParCoupons = original_par_coupon_setting

        vars_ = self.CommonVars() # Renamed to avoid conflict with built-in 'vars'

        vars_.today = ql.Date(15, ql.February, 2002)
        ql.Settings.instance().evaluationDate = vars_.today

        vars_.settlement = ql.Date(19, ql.February, 2002)
        # flat yield term structure implying 1x5 swap at 5%
        vars_.termStructure.linkTo(ql.FlatForward(vars_.settlement,
                                                  0.04875825,
                                                  ql.Actual365Fixed()))

        atmRate = vars_.makeSwap(0.0).fairRate()

        itmSwap = vars_.makeSwap(0.8 * atmRate)
        atmSwap = vars_.makeSwap(atmRate)
        otmSwap = vars_.makeSwap(1.2 * atmRate)

        a = 0.048696
        sigma = 0.0058904
        model = ql.HullWhite(vars_.termStructure, a, sigma)

        exerciseDates = []
        leg = atmSwap.fixedLeg()
        for cf in leg:
            # coupon = ql.as_coupon(cf) # Old way, might not work for all coupon types
            coupon = ql.as_coupon(cf) if hasattr(ql, 'as_coupon') else ql.Coupon.from_cashflow(cf)
            if coupon:
                exerciseDates.append(coupon.accrualStartDate())

        exercise = ql.BermudanExercise(exerciseDates)

        treeEngine = ql.TreeSwaptionEngine(model, 50)
        fdmEngine = ql.FdHullWhiteSwaptionEngine(model)

        itmValue, atmValue, otmValue = 0.0, 0.0, 0.0
        itmValueFdm, atmValueFdm, otmValueFdm = 0.0, 0.0, 0.0

        if not usingAtParCoupons:
            itmValue    = 42.2402;    atmValue = 12.9032;    otmValue = 2.49758
            itmValueFdm = 42.2111; atmValueFdm = 12.8879; otmValueFdm = 2.44443
        else:
            itmValue    = 42.2460;    atmValue = 12.9069;    otmValue = 2.4985
            itmValueFdm = 42.2091; atmValueFdm = 12.8864; otmValueFdm = 2.4437

        tolerance = 1.0e-4

        swaption = ql.Swaption(itmSwap, exercise)
        swaption.setPricingEngine(treeEngine)
        self.assertAlmostEqual(swaption.NPV(), itmValue, delta=tolerance,
                               msg=(f"failed to reproduce cached in-the-money swaption value (Tree):\n"
                                    f"calculated: {swaption.NPV()}\nexpected:   {itmValue}"))
        swaption.setPricingEngine(fdmEngine)
        self.assertAlmostEqual(swaption.NPV(), itmValueFdm, delta=tolerance,
                               msg=(f"failed to reproduce cached in-the-money swaption value (FDM):\n"
                                    f"calculated: {swaption.NPV()}\nexpected:   {itmValueFdm}"))

        swaption = ql.Swaption(atmSwap, exercise)
        swaption.setPricingEngine(treeEngine)
        self.assertAlmostEqual(swaption.NPV(), atmValue, delta=tolerance,
                               msg=(f"failed to reproduce cached at-the-money swaption value (Tree):\n"
                                    f"calculated: {swaption.NPV()}\nexpected:   {atmValue}"))

        swaption.setPricingEngine(fdmEngine)
        self.assertAlmostEqual(swaption.NPV(), atmValueFdm, delta=tolerance,
                               msg=(f"failed to reproduce cached at-the-money swaption value (FDM):\n"
                                    f"calculated: {swaption.NPV()}\nexpected:   {atmValueFdm}"))

        swaption = ql.Swaption(otmSwap, exercise)
        swaption.setPricingEngine(treeEngine)
        self.assertAlmostEqual(swaption.NPV(), otmValue, delta=tolerance,
                               msg=(f"failed to reproduce cached out-of-the-money swaption value (Tree):\n"
                                    f"calculated: {swaption.NPV()}\nexpected:   {otmValue}"))

        swaption.setPricingEngine(fdmEngine)
        self.assertAlmostEqual(swaption.NPV(), otmValueFdm, delta=tolerance,
                               msg=(f"failed to reproduce cached out-of-the-money swaption value (FDM):\n"
                                    f"calculated: {swaption.NPV()}\nexpected:   {otmValueFdm}"))

        modified_exerciseDates = []
        for d in exerciseDates:
            modified_exerciseDates.append(vars_.calendar.adjust(d - ql.Period(10, ql.Days)))

        exercise = ql.BermudanExercise(modified_exerciseDates)

        if not usingAtParCoupons:
            itmValue = 42.1791; atmValue = 12.7699; otmValue = 2.4368
        else:
            itmValue = 42.1849; atmValue = 12.7736; otmValue = 2.4379

        swaption = ql.Swaption(itmSwap, exercise)
        swaption.setPricingEngine(treeEngine)
        self.assertAlmostEqual(swaption.NPV(), itmValue, delta=tolerance,
                               msg=(f"failed to reproduce cached in-the-money swaption value (Tree, modified exercise):\n"
                                    f"calculated: {swaption.NPV()}\nexpected:   {itmValue}"))

        swaption = ql.Swaption(atmSwap, exercise)
        swaption.setPricingEngine(treeEngine)
        self.assertAlmostEqual(swaption.NPV(), atmValue, delta=tolerance,
                               msg=(f"failed to reproduce cached at-the-money swaption value (Tree, modified exercise):\n"
                                    f"calculated: {swaption.NPV()}\nexpected:   {atmValue}"))

        swaption = ql.Swaption(otmSwap, exercise)
        swaption.setPricingEngine(treeEngine)
        self.assertAlmostEqual(swaption.NPV(), otmValue, delta=tolerance,
                               msg=(f"failed to reproduce cached out-of-the-money swaption value (Tree, modified exercise):\n"
                                    f"calculated: {swaption.NPV()}\nexpected:   {otmValue}"))
        # Restore setting if it was ever changed (though not in this test)
        # ql.IborCoupon.Settings.instance().setUsingAtParCoupons(original_par_coupon_setting)


    @if_speed('Fast')
    def testCachedG2Values(self):
        print("Testing Bermudan swaption with G2 model against cached values...")

        usingAtParCoupons = ql.IborCoupon.Settings.instance().usingAtParCoupons()

        vars_ = self.CommonVars()

        vars_.today = ql.Date(15, ql.September, 2016)
        ql.Settings.instance().evaluationDate = vars_.today
        vars_.settlement = ql.Date(19, ql.September, 2016)

        vars_.termStructure.linkTo(ql.FlatForward(vars_.settlement,
                                                  0.04875825,
                                                  ql.Actual365Fixed()))

        atmRate = vars_.makeSwap(0.0).fairRate()

        swaptions = []
        strikes_mult = [s/100.0 for s in range(50, 151, 25)] # 0.5, 0.75, 1.0, 1.25, 1.50

        for s_mult in strikes_mult:
            swap = vars_.makeSwap(s_mult * atmRate)
            exerciseDates = []
            for cf in swap.fixedLeg():
                # coupon = ql.as_coupon(cf) # Old way
                coupon = ql.as_coupon(cf) if hasattr(ql, 'as_coupon') else ql.Coupon.from_cashflow(cf)
                if coupon:
                    exerciseDates.append(coupon.accrualStartDate())

            exercise = ql.BermudanExercise(exerciseDates)
            swaptions.append(ql.Swaption(swap, exercise))

        a = 0.1; sigma = 0.01; b = 0.2; eta = 0.013; rho = -0.5

        g2Model = ql.G2(vars_.termStructure, a, sigma, b, eta, rho)

        fdmEngine = ql.FdG2SwaptionEngine(g2Model, 50, 75, 75, 0, 1e-3)
        treeEngine = ql.TreeSwaptionEngine(g2Model, 50)

        expectedFdm = [0.0] * 5
        expectedTree = [0.0] * 5

        if not usingAtParCoupons:
            tmpExpectedFdm  = [ 103.231, 54.6519, 20.0475, 5.26941, 1.07097 ]
            tmpExpectedTree = [ 103.245, 54.6685, 20.1656, 5.43999, 1.12702 ]
            expectedFdm = tmpExpectedFdm[:]
            expectedTree = tmpExpectedTree[:]
        else:
            tmpExpectedFdm  = [ 103.227, 54.6502, 20.0469, 5.26924, 1.07093 ]
            tmpExpectedTree = [ 103.248, 54.6726, 20.1685, 5.44118, 1.12737 ]
            expectedFdm = tmpExpectedFdm[:]
            expectedTree = tmpExpectedTree[:]

        tol = 0.005
        for i in range(len(swaptions)):
            swaptions[i].setPricingEngine(fdmEngine)
            calculatedFdm = swaptions[i].NPV()

            self.assertAlmostEqual(calculatedFdm, expectedFdm[i], delta=tol,
                                   msg=(f"failed to reproduce cached G2 FDM swaption value (strike mult {strikes_mult[i]}):\n"
                                        f"calculated: {calculatedFdm:.5f}\nexpected:   {expectedFdm[i]:.5f}"))

            swaptions[i].setPricingEngine(treeEngine)
            calculatedTree = swaptions[i].NPV()
            self.assertAlmostEqual(calculatedTree, expectedTree[i], delta=tol,
                                   msg=(f"failed to reproduce cached G2 Tree swaption value (strike mult {strikes_mult[i]}):\n"
                                        f"calculated: {calculatedTree:.5f}\nexpected:   {expectedTree[i]:.5f}"))

    def testTreeEngineTimeSnapping(self):
        print("Testing snap of exercise dates for discretized swaption...")

        today = ql.Date(8, ql.July, 2021)
        original_eval_date = ql.Settings.instance().evaluationDate
        ql.Settings.instance().evaluationDate = today

        termStructure = ql.RelinkableYieldTermStructureHandle()
        termStructure.linkTo(ql.FlatForward(today, 0.02, ql.Actual365Fixed()))
        index = ql.Euribor3M(termStructure)

        # The lambda in C++ is a local helper function. Define it as such in Python.
        def makeBermudanSwaption(callDate_param, idx_param, effectiveDate_param):
            # Chained MakeVanillaSwap call
            swap = (ql.MakeVanillaSwap(ql.Period(10, ql.Years), idx_param, 0.05)
                        .withEffectiveDate(effectiveDate_param)
                        .withNominal(10000.00)
                        .withType(ql.Swap.Payer))() # Call the maker object to get the swap

            exerciseDates = [effectiveDate_param, callDate_param]
            bermudanExercise = ql.BermudanExercise(exerciseDates)
            bermudanSwaption = ql.Swaption(swap, bermudanExercise)
            return bermudanSwaption

        intervalOfDaysToTest = 10

        initialCallDate = ql.Date(15, ql.May, 2030)
        # effectiveDate for the swap is fixed inside the helper in C++
        # but it's better to define it here for clarity if it's static.
        # The C++ lambda captures `effectiveDate` from its outer scope.
        effective_date_for_swap = ql.Date(15, ql.May, 2025)
        cal = index.fixingCalendar()

        for i in range(-intervalOfDaysToTest, intervalOfDaysToTest + 1):
            callDate = initialCallDate + ql.Period(i, ql.Days)

            if cal.isBusinessDay(callDate):
                bermudanSwaption = makeBermudanSwaption(callDate, index, effective_date_for_swap)

                model = ql.HullWhite(termStructure)

                bermudanSwaption.setPricingEngine(ql.FdHullWhiteSwaptionEngine(model))
                npvFD = bermudanSwaption.NPV()

                timesteps = 14 * 4 * 4

                bermudanSwaption.setPricingEngine(ql.TreeSwaptionEngine(model, timesteps))
                npvTree = bermudanSwaption.NPV()

                npvDiff = npvTree - npvFD
                tolerance = 1.0

                self.assertLessEqual(abs(npvDiff), tolerance,
                                     msg=(f"At {callDate.ISO()}: The difference between NPV of FD ({npvFD:.2f}) and Tree ({npvTree:.2f}) "
                                          f"engine is {npvDiff:.2f}, expected to be <= {tolerance:.2f}."))

        ql.Settings.instance().evaluationDate = original_eval_date # Restore global setting


if __name__ == '__main__':
    print("Presolve testQuantLib.py ...") # Mimic C++ test runner message
    unittest.main(argv=['first-arg-is-ignored'], exit=False)